# Etract, Transform, Load

**Goal**: Exploratory Data Analysis - to design the experiment. Transform the data and load it back to the database.

Specific:

- Extract applicant information regarding addmissions quiz completion.
- Design a research question, null hypothesis and alternative hypothesis for the experiment.
- Create functions for transforming applicant documents and loading them to a database.
- Build a Python class to streamline the experiment.

In [6]:
import random

import pandas as pd
from pymongo import MongoClient
